<a href="https://colab.research.google.com/github/kakkia/final_project/blob/main/saved_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from pathlib import Path
import numpy as np
import joblib
from keras.preprocessing import image

import matplotlib.pyplot as plt
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [1]:
import pathlib

data = "/content/drive/MyDrive/dataset"
train_ds = pathlib.Path(data)
validation = "/content/drive/MyDrive/validation"
val_ds = pathlib.Path(validation)

In [3]:
batch_size = 32
img_height = 180
img_width = 180

In [4]:
train_images = tf.keras.utils.image_dataset_from_directory(
  train_ds,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
  )


Found 1007 files belonging to 3 classes.
Using 806 files for training.


In [5]:
test_images = tf.keras.utils.image_dataset_from_directory(
  train_ds,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
  )

Found 1007 files belonging to 3 classes.
Using 201 files for validation.


In [6]:
class_names = train_images.class_names
print(class_names)

['perfect', 'shoo', 'what']


In [7]:
normalization_layer = layers.Rescaling(1./255)

normalized_ds = train_images.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.21960786 1.0


In [8]:
# Define a simple sequential model

def create_model():
  model = Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(img_height, img_width, 3)),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(3)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 180, 180, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 90, 90, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 90, 90, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 45, 45, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 45, 45, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 22, 22, 64)       0

In [9]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          epochs=10,
          validation_data=(test_images),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
26/26 [==============================] - ETA: 0s - loss: 145.4824 - sparse_categorical_accuracy: 0.4442
Epoch 1: saving model to training_1/cp.ckpt
26/26 [==============================] - 111s 4s/step - loss: 145.4824 - sparse_categorical_accuracy: 0.4442 - val_loss: 0.7220 - val_sparse_categorical_accuracy: 0.7363
Epoch 2/10
26/26 [==============================] - ETA: 0s - loss: 0.4704 - sparse_categorical_accuracy: 0.8114
Epoch 2: saving model to training_1/cp.ckpt
26/26 [==============================] - 24s 911ms/step - loss: 0.4704 - sparse_categorical_accuracy: 0.8114 - val_loss: 0.5684 - val_sparse_categorical_accuracy: 0.8308
Epoch 3/10
26/26 [==============================] - ETA: 0s - loss: 0.4525 - sparse_categorical_accuracy: 0.8201
Epoch 3: saving model to training_1/cp.ckpt
26/26 [==============================] - 24s 902ms/step - loss: 0.4525 - sparse_categorical_accuracy: 0.8201 - val_loss: 0.6271 - val_sparse_categorical_accuracy: 0.7015
Epoch 4/10
26/26 

In [10]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.index', 'cp.ckpt.data-00000-of-00001']

In [11]:
# Now rebuild a fresh, untrained model and evaluate it on the test set. An untrained model will perform at chance levels (~10% accuracy):

#Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

7/7 - 2s - loss: 15.8519 - sparse_categorical_accuracy: 0.3781 - 2s/epoch - 297ms/step
Untrained model, accuracy: 37.81%


In [12]:
# Then load the weights from the checkpoint and re-evaluate:

#Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

7/7 - 2s - loss: 0.7376 - sparse_categorical_accuracy: 0.8159 - 2s/epoch - 287ms/step
Restored model, accuracy: 81.59%


In [13]:
model.save("/content/drive/MyDrive/my_model/hand_gestures.h5")

In [21]:
# Train a new model, and save uniquely named checkpoints once every five epochs:

#Include the epoch in the file name (uses `str.format`)

checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images),
          verbose=0)


Epoch 7: saving model to training_2/cp-0007.ckpt

Epoch 13: saving model to training_2/cp-0013.ckpt

Epoch 19: saving model to training_2/cp-0019.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 31: saving model to training_2/cp-0031.ckpt

Epoch 37: saving model to training_2/cp-0037.ckpt

Epoch 44: saving model to training_2/cp-0044.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


In [22]:
# look at resulting checkpoints 

os.listdir(checkpoint_dir)

['cp-0000.ckpt.data-00000-of-00001',
 'cp-0031.ckpt.data-00000-of-00001',
 'cp-0019.ckpt.data-00000-of-00001',
 'cp-0019.ckpt.index',
 'cp-0037.ckpt.index',
 'checkpoint',
 'cp-0007.ckpt.index',
 'cp-0007.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0037.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0044.ckpt.index',
 'cp-0013.ckpt.index',
 'cp-0000.ckpt.index',
 'cp-0013.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0044.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'cp-0031.ckpt.index']

In [23]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [24]:
# To test, reset the model and load the latest checkpoint:

# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

7/7 - 3s - loss: 0.7772 - sparse_categorical_accuracy: 0.8955 - 3s/epoch - 359ms/step
Restored model, accuracy: 89.55%


In [26]:
model.save('/content/drive/MyDrive/my_model')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/my_model/assets
